In [1]:
from storefact import get_store_from_url
from kartothek.io.eager import read_table
from functools import partial
from kartothek.core.dataset import DatasetMetadata
import pandas as pd


In [34]:
import json
with open('application.cfg' ,'r') as fh:
    config = json.load(fh)

opd_store_url = config["storage_urls"]["retrace_procurement_decision"]
opd_store_uuid = "opd_data_opd_002"
opd_store_fact = partial(get_store_from_url, opd_store_url)

opp_url = config["storage_urls"]["opd_predictions"]
opp_store_uuid = "wfc_prod_opd_pred_data_opd_002"
opp_store_fact = partial(get_store_from_url, opp_url)

opx_url = config["storage_urls"]["opd_expected_deliveries"]
opx_store_uuid = "wfc_prod_opd_exp_del_data_opd_002"
opx_store_fact = partial(get_store_from_url, opx_url)

In [35]:

print(DatasetMetadata.load_from_store(opd_store_uuid, opd_store_fact()))
print(DatasetMetadata.load_from_store(opx_store_uuid, opx_store_fact()))


DatasetMetadata(uuid=opd_data_opd_002, tables=['procurement'], partition_keys=['ppy_external_code', 'reference_date', 'run_timestamp'], metadata_version=4, indices=['p_external_code'], explicit_partitions=True)
DatasetMetadata(uuid=wfc_prod_opd_exp_del_data_opd_002, tables=['expected_deliveries'], partition_keys=['ppy_external_code', 'reference_date', 'run_timestamp'], metadata_version=4, indices=['p_external_code'], explicit_partitions=True)


In [21]:
import datetime
predicates = [[
    ("reference_date", "==", datetime.date(2020,8,28)),
    ("l_external_code", "in", ['162']),
    ("p_external_code", "in", ['7047013777'])
]]
print(predicates)
table_name = "expected_deliveries"

[[('reference_date', '==', datetime.date(2020, 8, 28)), ('l_external_code', 'in', ['162']), ('p_external_code', 'in', ['7047013777'])]]


In [22]:
%%time
df_x = read_table(
    store=opx_store_fact,
    dataset_uuid=opx_store_uuid,
    table=table_name,
    predicates=predicates
)
print(df_x.shape)
df_x.head()

(9, 11)
CPU times: user 935 ms, sys: 97.8 ms, total: 1.03 s
Wall time: 1.23 s


In [23]:
df_x.head()

,ppy_external_code,reference_date,run_timestamp,availability_date,delivery_date,l_external_code,order_amount_arrived_before_stocks_su,order_amount_arrived_su,order_amount_su,p_external_code,shelf_life
0,STANDARD,2020-08-28,1598592940,2020-07-15,2020-07-15,162,8.0,8.0,0.0,7047013777,50.0
1,STANDARD,2020-08-28,1598592940,2020-07-20,2020-07-20,162,8.0,8.0,0.0,7047013777,50.0
2,STANDARD,2020-08-28,1598592940,2020-07-22,2020-07-22,162,8.0,8.0,0.0,7047013777,50.0
3,STANDARD,2020-08-28,1598592940,2020-08-01,2020-08-01,162,8.0,8.0,0.0,7047013777,50.0
4,STANDARD,2020-08-28,1598592940,2020-08-06,2020-08-06,162,8.0,8.0,0.0,7047013777,50.0


In [24]:
%%time
df_opd = read_table(
    store=opd_store_fact,
    dataset_uuid=opd_store_uuid,
    table="procurement",
    predicates=predicates
)
print(df_opd.shape)
df_opd.head()

(5, 56)
CPU times: user 949 ms, sys: 142 ms, total: 1.09 s
Wall time: 1.22 s


In [25]:
df_opd.head()

,ppy_external_code,reference_date,run_timestamp,availability_time,base_unit_id,dc_order_quantity_su,delivery_time,l_external_code,l_id,l_id_source,...,stock_build_event_upto,stocks_su,strategy_info,supplier_external_code,third_party_demand_su,uc_factor,unconstrained_order_projection_sum_per_dc_su,unconstrained_order_proposal_su,unconstrained_order_proposal_sum_per_dc_su,unconstrained_projection_su
0,STANDARD,2020-08-28,1598592940,2020-08-31 10:00:00,PCS,NaN,2020-08-31 10:00:00,162,2409,2024,...,None,7.0,"{""version"":""v1"",""float_parameters"":{""manual_de...",W9,NaN,1.0,NaN,NaN,NaN,NaN
1,STANDARD,2020-08-28,1598592940,2020-09-02 10:00:00,PCS,NaN,2020-09-02 10:00:00,162,2409,2024,...,None,11.0,"{""version"":""v1"",""float_parameters"":{""manual_de...",W9,NaN,1.0,NaN,NaN,NaN,NaN
2,STANDARD,2020-08-28,1598592940,2020-09-03 10:00:00,PCS,NaN,2020-09-03 10:00:00,162,2409,2024,...,None,10.0,"{""version"":""v1"",""float_parameters"":{""manual_de...",W9,NaN,1.0,NaN,NaN,NaN,NaN
3,STANDARD,2020-08-28,1598592940,2020-09-04 10:00:00,PCS,NaN,2020-09-04 10:00:00,162,2409,2024,...,None,9.0,"{""version"":""v1"",""float_parameters"":{""manual_de...",W9,NaN,1.0,NaN,NaN,NaN,NaN
4,STANDARD,2020-08-28,1598592940,2020-09-05 22:00:00,PCS,NaN,2020-09-05 22:00:00,162,2409,2024,...,None,8.0,"{""version"":""v1"",""float_parameters"":{""manual_de...",W9,NaN,1.0,NaN,NaN,NaN,NaN


In [26]:
df_opd.head().transpose()

,0,1,2,3,4
ppy_external_code,STANDARD,STANDARD,STANDARD,STANDARD,STANDARD
reference_date,2020-08-28 00:00:00,2020-08-28 00:00:00,2020-08-28 00:00:00,2020-08-28 00:00:00,2020-08-28 00:00:00
run_timestamp,1598592940,1598592940,1598592940,1598592940,1598592940
availability_time,2020-08-31 10:00:00,2020-09-02 10:00:00,2020-09-03 10:00:00,2020-09-04 10:00:00,2020-09-05 22:00:00
base_unit_id,PCS,PCS,PCS,PCS,PCS
dc_order_quantity_su,NaN,NaN,NaN,NaN,NaN
delivery_time,2020-08-31 10:00:00,2020-09-02 10:00:00,2020-09-03 10:00:00,2020-09-04 10:00:00,2020-09-05 22:00:00
l_external_code,162,162,162,162,162
l_id,2409,2409,2409,2409,2409
l_id_source,2024,2024,2024,2024,2024


In [1]:
#Code for real task 

from kartothek.io.dask.dataframe import read_dataset_as_ddf, update_dataset_from_ddf
from storefact import get_store_from_url
from kartothek.io.eager import read_table
from functools import partial
from kartothek.core.dataset import DatasetMetadata

dataset_uuid_stag = "mor_staging_dask_dplc_data_002"
dataset_uuid_prod = "mor_prod_dplc_data_003"

store_url_prod = "hazure://nckqjylgla4qm:se%3D2020-11-30T00%253A00%253A00Z%26sp%3Drl%26sv%3D2019-02-02%26sr%3Dc%26sig%3DTXsi2eekoiIGAhLn1AqknBJHjSJLKwzPQCu%2FOUHa854%253D@mor-prod-demand-predictions-lastcalcs?use_sas=true&create_if_missing=false"
store_url_stag = "hazure://nckqjylgla4qm:se%3D2020-11-30T00%253A00%253A00Z%26sp%3Drl%26sv%3D2019-02-02%26sr%3Dc%26sig%3DYn90Kf9mkwNVecSV7kzfv7Pbc0nfnAprfrkVd59pXpY%253D@mor-staging-dask-demand-predictions-lastcalcs?use_sas=true&create_if_missing=false"

store_prod = partial(get_store_from_url, store_url_prod)
store_stag = partial(get_store_from_url, store_url_stag)

table = "predictions"

print(DatasetMetadata.load_from_store(dataset_uuid_prod, store_prod()))
#print(DatasetMetadata.load_from_store(dataset_uuid_stag, store_stag()))

# read prod
ddf = read_dataset_as_ddf(dataset_uuid_prod,
                        store_prod, table)



# write to staging
graph = update_dataset_from_ddf(ddf, store_stag,
                         dataset_uuid_stag, table) 
graph.compute()

ModuleNotFoundError: No module named 'azure'

In [ ]:
#Code for experimentation  

from kartothek.io.dask.dataframe import read_dataset_as_ddf, update_dataset_from_ddf
from storefact import get_store_from_url
from kartothek.io.eager import read_table
from functools import partial
from kartothek.core.dataset import DatasetMetadata

dataset_uuid_stag = "mor_staging_dask_dplc_data_002"
dataset_uuid_prod = "mor_prod_dplc_data_003"

store_url_prod = "hazure://nckqjylgla4qm:se%3D2020-11-30T00%253A00%253A00Z%26sp%3Drl%26sv%3D2019-02-02%26sr%3Dc%26sig%3DTXsi2eekoiIGAhLn1AqknBJHjSJLKwzPQCu%2FOUHa854%253D@mor-prod-demand-predictions-lastcalcs?use_sas=true&create_if_missing=false"
store_url_stag = "hazure://nckqjylgla4qm:se%3D2020-11-30T00%253A00%253A00Z%26sp%3Drl%26sv%3D2019-02-02%26sr%3Dc%26sig%3DYn90Kf9mkwNVecSV7kzfv7Pbc0nfnAprfrkVd59pXpY%253D@mor-staging-dask-demand-predictions-lastcalcs?use_sas=true&create_if_missing=false"

store_prod = partial(get_store_from_url, store_url_prod)
store_stag = partial(get_store_from_url, store_url_stag)

table = "predictions"

print(DatasetMetadata.load_from_store(dataset_uuid_prod, store_prod()))
#print(DatasetMetadata.load_from_store(dataset_uuid_stag, store_stag()))

# read staging
ddf = read_dataset_as_ddf(dataset_uuid_stag,
                        store_stag, table)



# write to prod
graph = update_dataset_from_ddf(ddf, store_prod,
                         dataset_uuid_prod, table) 
graph.compute()